In [1]:
# import libraries
import pandas as pd

In [2]:
# read in the data
data_df = pd.read_csv('../data/processed_csv/project3_raw_final.csv')
data_df

,AGE2,IRMARIT,HEALTH,MOVSINPYR2,SEXATRACT,SEXIDENT,DIFFTHINK,IREDUHIGHST2,WRKSTATWK2,IRWRKSTAT,...,MJEVER,HEREVER,METHAMEVR,PNRANYLIF,TRQANYLIF,STMANYLIF,SEDANYLIF,halluciogens,inhalants,not_mental
0,15,1,2.0,0.0,1.0,1.0,2.0,11,1.0,1,...,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0,1,1
1,17,1,2.0,0.0,1.0,1.0,2.0,11,1.0,1,...,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0,0,0
2,14,4,2.0,0.0,1.0,1.0,2.0,11,1.0,1,...,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0,0,0
3,8,4,2.0,1.0,1.0,1.0,2.0,9,4.0,3,...,1.0,2.0,2.0,2.0,1.0,1.0,2.0,0,0,0
4,14,1,2.0,0.0,1.0,1.0,2.0,11,1.0,1,...,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120538,15,1,2.0,0.0,1.0,1.0,2.0,9,3.0,1,...,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1,0,0
120539,10,4,4.0,3.0,2.0,1.0,1.0,9,1.0,1,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1,0,1
120540,16,4,2.0,0.0,5.0,2.0,2.0,11,1.0,1,...,2.0,2.0,2.0,1.0,2.0,2.0,1.0,0,1,0
120541,12,4,3.0,1.0,1.0,1.0,2.0,8,3.0,1,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1,0,0


In [3]:
# Assign X (data) and y (target)
X = data_df.drop(columns=['CG30EST',
 'ALCDAYS',
 'COCEVER',
 'CRKEVER',
 'MJEVER',
 'HEREVER',
 'METHAMEVR',
 'PNRANYLIF',
 'TRQANYLIF',
 'STMANYLIF',
 'SEDANYLIF', 
 'halluciogens', 
 'inhalants',
 'not_mental'])
y = data_df['PNRANYLIF'].values
print(X.shape, y.shape)

(120543, 15) (120543,)


In [4]:
from sklearn.preprocessing import LabelEncoder
# Step 1: Label-encode data set
encoded_y = LabelEncoder().fit(y).transform(y)

In [5]:
codes_df = pd.DataFrame({"Original class": y, "Encoded label": encoded_y})

In [6]:
# show original clas with encoded labels
codes_df.drop_duplicates()


,Original class,Encoded label
0,1.0,0
3,2.0,1


In [7]:
# Use sklearn's `train_test_split` to split the data into training and testing
from sklearn.model_selection import train_test_split

# split is random (shuffled) automatically
X_train_test, X_val, y_train_test, y_val = train_test_split(X, encoded_y, random_state=42, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, random_state=42, test_size=0.2)

# Logistic Regression

In [8]:
# Create a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=100000)

In [9]:
# Fit (train) or model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=100000)

In [10]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
# print(f"Validation Data Score: {classifier.score(X_val, y_val)}")

Training Data Score: 0.6673464684871528
Testing Data Score: 0.6652226011613973


In [11]:
# make a prediction on the validation data before adjusting parameters
predictions = classifier.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,1
4,1,1
5,0,0
6,0,0
7,0,0
8,1,1
9,0,1


In [12]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'penalty': ['L1', 'L2', 'none']}
grid = GridSearchCV(classifier, param_grid, verbose=True, n_jobs=-2)

In [13]:
# Fit the model using the grid search estimator. 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=LogisticRegression(max_iter=100000), n_jobs=-2,
             param_grid={'C': [1, 5, 10, 50], 'penalty': ['L1', 'L2', 'none']},
             verbose=True)

In [14]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'penalty': 'none'}


In [15]:
# List the best score
print(grid.best_score_)

0.667415600875677


In [16]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {grid.score(X_train, y_train)}")
print(f"Testing Data Score: {grid.score(X_test, y_test)}")
# print(f"Validation Data Score: {grid.score(X_val, y_val)}")

Training Data Score: 0.667519299458463
Testing Data Score: 0.6644852060097705


In [17]:
# make a prediction on the validation data
predictions = grid.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,1
4,1,1
5,0,0
6,0,0
7,0,0
8,1,1
9,0,1


In [18]:
import joblib
# save/ export the model
joblib.dump(grid, '../Machine_learning_models/Drugs_LogRegr_final_pnr.smd')
print("Model is saved.")

Model is saved.


In [19]:
load_model = joblib.load('../Machine_learning_models/Drugs_LogRegr_final_pnr.smd')

In [20]:
load_model.predict(X_val)

array([0, 0, 0, ..., 0, 1, 0])

# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [22]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 0.9379421592349348
Testing Data Score: 0.6409807355516638


In [23]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_

# We can sort the features by their importance
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.13782599911005147, 'AGE2'),
 (0.12454017095608659, 'ALCUS30D'),
 (0.11392796796300743, 'IRPINC3'),
 (0.11208385265320071, 'CIGTRY'),
 (0.09862304100045684, 'IREDUHIGHST2'),
 (0.08025152275901404, 'HEALTH'),
 (0.0633772628759932, 'WRKNUMJOB2'),
 (0.058101937230030744, 'MOVSINPYR2'),
 (0.04799015922839169, 'WRKSTATWK2'),
 (0.042460501603604345, 'IRMARIT'),
 (0.03533016814457879, 'SEXATRACT'),
 (0.029584393528185867, 'CIG100LF'),
 (0.02812974862392026, 'IRWRKSTAT'),
 (0.01468751563026621, 'SEXIDENT'),
 (0.013085758693211855, 'DIFFTHINK')]

In [24]:
# make a prediction on the validation data
rf_predictions = rf.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,1
4,1,1
5,0,0
6,0,0
7,0,0
8,1,1
9,0,1


In [25]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators': [100, 200, 500],
                'max_features': [10, 20, 50]}
grid_rf = GridSearchCV(rf, rf_param_grid, verbose=True, n_jobs=-2)

In [26]:
# Fit the model using the grid search estimator. 
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-2,
             param_grid={'max_features': [10, 20, 50],
                         'n_estimators': [100, 200, 500]},
             verbose=True)

In [27]:
# List the best parameters for this dataset
print(grid_rf.best_params_)

{'max_features': 10, 'n_estimators': 500}


In [28]:
# List the best score
print(grid_rf.best_score_)

0.6380228136882129


In [29]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {grid_rf.score(X_train, y_train)}")
print(f"Testing Data Score: {grid_rf.score(X_test, y_test)}")
# print(f"Validation Data Score: {grid.score(X_val, y_val)}")

Training Data Score: 0.9379421592349348
Testing Data Score: 0.6373859341874827


In [30]:
# make a prediction on the validation data
predictions_rf = grid_rf.predict(X_val)
pd.DataFrame({"Prediction": predictions_rf, "Actual": y_val}).head(15)

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,1,1
4,0,1
5,0,0
6,0,0
7,0,0
8,1,1
9,0,1


In [31]:
# save/ export the model
joblib.dump(grid_rf, '../Machine_learning_models/Drugs_RandomForest_final_pnr.smd')
print("Model is saved.")

Model is saved.


In [32]:
load_model_rf = joblib.load('../Machine_learning_models/Drugs_RandomForest_final_pnr.smd')

In [33]:
load_model_rf.predict(X_val)

array([0, 0, 0, ..., 0, 0, 1])